# Librerias comunes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importacion datos

In [ ]:
from mongo_local_module import connect_to_db, get_collection_as_dataframe

## Conectar a db

In [ ]:
db_name = 'ds_market'         # Nombre de la base de datos

# Conectar a la base de datos
db = connect_to_db(db_name, local=True)

## Obtener colecciones

In [ ]:
# Diccionario colecciones
db_collections = [
    'calendar',
    'items',
    'prices',
    'tiendas',
    'departments',
    'sales_main',
    'weekly_sales',
    'year_sales',
    'day_sales'
    ]

In [ ]:
df_cal = get_collection_as_dataframe(db, 'calendar').drop(columns='_id')
# Se añadieron columnas week y month a calendar para hacer cruces de datos mas faciles

df_cal = df_cal.fillna('None') # event tiene NaN

display(df_cal.head())

In [ ]:
df_shops = get_collection_as_dataframe(db, 'tiendas').drop(columns='_id')
df_items = get_collection_as_dataframe(db, 'items').drop(columns='_id')

df_ws = get_collection_as_dataframe(db, 'weekly_sales').drop(columns='_id')

# Añadimos eventos para no perder esa información
# Crear un diccionario desde df_cal
event_dict = df_cal.set_index('week')['event'].to_dict()

# Añadir la columna 'event' a df_ws usando el método 'map'
df_ws['event'] = df_ws['week'].map(event_dict)

# Clustering productos

## Creacion caracteristicas

In [ ]:
import feature_creation_module as fcm

In [ ]:
# DataFrames de entrada
feature_matrix = fcm.create_features(df_items, df_shops, df_ws)

# Renombrar características si es necesario
feature_matrix_renamed = fcm.rename_features(feature_matrix)

**Desviación estándar de las semanas de venta (`STD(sales.week)`):**  

Esta métrica revela si las ventas de un producto están concentradas en ciertos períodos del año o dispersas. Un alto valor indica ventas distribuidas, mientras que un valor bajo sugiere un patrón estacional claro. Esto es útil para identificar productos estacionales.

**Media de los años de venta (`MEAN(sales.year)`):**  

La media de los años de venta muestra cuándo los productos han sido más populares, ayudando a detectar si una tendencia es reciente o si un producto está en declive. Esto facilita el agrupamiento de productos según su ciclo de vida y relevancia actual.

**Media de las semanas de venta (`MEAN(sales.week)`):**  

Esta métrica indica en qué parte del año un producto suele venderse más. Permite agrupar productos según su popularidad en diferentes épocas, ayudando a identificar patrones estacionales y planificar estrategias de ventas e inventario.

In [ ]:
feature_matrix_renamed.head(10)

## Preprocesamiento de los datos

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.exceptions import NotFittedError

def preprocess_features(feature_matrix):
    try:
        # Identificar columnas categóricas y numéricas
        categorical_columns = feature_matrix.select_dtypes(include=['object', 'category']).columns.tolist()
        numeric_columns = feature_matrix.select_dtypes(include=[float, int]).columns.tolist()

        print(f'Cat cols: {len(categorical_columns)}')
        print(f'Num cols: {len(numeric_columns)}')

        # Definir la lista de transformadores
        transformers = []
        
        # Agregar el RobustScaler para las columnas numéricas si existen
        if numeric_columns:
            transformers.append(('num', RobustScaler(), numeric_columns))
        
        # Agregar el OneHotEncoder para las columnas categóricas si existen
        if categorical_columns:
            transformers.append(('cat', OneHotEncoder(drop='first', sparse=False), categorical_columns))

        # Crear el preprocesador con los transformadores existentes
        preprocessor = ColumnTransformer(transformers)

        # Aplicar las transformaciones utilizando el preprocesador
        scaled_features = preprocessor.fit_transform(feature_matrix)

        # Obtener el nombre de las columnas después de la transformación
        all_columns = numeric_columns.copy()  # Empezamos con las columnas numéricas

        if categorical_columns:
            encoded_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)
            all_columns += encoded_columns.tolist()

        # Convertir el resultado de nuevo a un DataFrame
        scaled_df = pd.DataFrame(scaled_features, columns=all_columns)

        # Devolver el DataFrame transformado
        return scaled_df
    
    except NotFittedError as e:
        print(f"Error: {e}")
        return None

# Uso
# scaled_df = preprocess_features(fm_selected_sales)

In [ ]:
scaled_df = preprocess_features(feature_matrix_renamed)

In [ ]:
scaled_df.head()

## Kmeans

In [ ]:
import kmeans_module as km_m

### Elbow & Silouette

In [ ]:
km_m.plot_elbow_silhouette(scaled_df, 20)

In [ ]:
optimal_k = 10 # Segun silueta

kmeans_model, feature_matrix_with_clusters = km_m.apply_kmeans_and_plot(optimal_k, scaled_df, feature_matrix_renamed)

feature_matrix_with_clusters.head(5)

### Analisis

#### PCAs

In [ ]:
pca_df = km_m.pca_visualization_2d(feature_matrix_with_clusters)

display(pca_df.head())

In [ ]:
km_m.pca_variance_plot(feature_matrix_with_clusters)

#### Importancia caracteristicas

In [ ]:
# Preprocesamos datos
cluster_col='Cluster'
cols_to_scale = feature_matrix_with_clusters.columns.tolist()
df_clusters = feature_matrix_with_clusters[[cluster_col]].copy().reset_index(drop=True)
cols_to_scale.remove(cluster_col) # Columna Objetivo
preprocessed_df = preprocess_features(feature_matrix_with_clusters[cols_to_scale])

# Combinar los DataFrames basándose en la columna 'Index'
preprocessed_df = pd.concat([preprocessed_df, df_clusters], axis=1)

clusters = kmeans_model.labels_

# sales_importance_df = km_m.get_feature_importances(feature_matrix_with_clusters, clusters, imp_threshold=0.05)
sales_importance_df = km_m.get_feature_importances(preprocessed_df, clusters, imp_threshold=0.05)

display(sales_importance_df.head(10))

### Dimension Redux

In [ ]:
pca_df, importance_df = km_m.redux_dimensions_pca_and_cluster(preprocessed_df, n_clusters=optimal_k, n_components=1)

In [ ]:
pca_df, importance_df = km_m.redux_dimensions_pca_and_cluster(preprocessed_df, n_clusters=optimal_k, n_components=2)

## CLUSTERING MODULE

In [ ]:
import clustering_module as cm

In [ ]:
# K-means
rf_model_kmeans, preprocessor_kmeans, pca_loadings_kmeans, pca_contributions_kmeans = cm.main(
    feature_matrix_renamed, method='kmeans', n_clusters=4
)

In [ ]:
# K-means
rf_model_kmeans, preprocessor_kmeans, pca_loadings_kmeans, pca_contributions_kmeans = cm.main(
    preprocessed_df, method='kmeans', n_clusters=10
)

In [ ]:
# Ejemplo con DBSCAN
rf_model_dbscan, preprocessor_dbscan, pca_loadings_dbscan, pca_contributions_dbscan = cm.main(
    feature_matrix_renamed, method='dbscan', eps=0.5, min_samples=5
)